# ENUNCIADO

- Implementar el detector de fondo naive usando la mediana como estimador. El algoritmo debe recibir el parámetro N (cantidad de frames utilizados para la estimación) y el intervalo de tiempo para recalcular el fondo.
- Se deben generar las mascaras de foreground y aplicarlas a los frames para segmentar los objetos en movimiento
- Comparar con alguno de los métodos vistos en la practica basados en mezcla de gaussianas

In [134]:
import numpy as np
import cv2 as cv
import copy

## Algoritmo de sustraccion de fondo mediante mezclas de Gaussianas (como se presentó en la práctica)

In [135]:
backSub = cv.createBackgroundSubtractorMOG2()

In [136]:
# Abrimos el archivo
#-------------------
filename = 'slow_traffic_small.mp4'
capture = cv.VideoCapture(filename)

fps = capture.get(cv.CAP_PROP_FPS)      # OpenCV v2.x used "CV_CAP_PROP_FPS"
frame_count = int(capture.get(cv.CAP_PROP_FRAME_COUNT))
duration = frame_count/fps
print(fps, frame_count, duration)

if not capture.isOpened:
    print('Falla al abrir el archivo: ' + filename)
    exit(0)

# Corremos la sustraccion
#------------------------
while True:
    # Leemos un frame
    ret, frame = capture.read()
    if frame is None:
        break
    
    # Aplicamos la sustracción al frame leído
    #----------------------------------------
    # Cada frame se utiliza tanto para calcular la máscara de primer plano como para actualizar el fondo.
    # Si se desea cambiar la tasa de aprendizaje utilizada para actualizar el modelo de fondo, es posible
    # establecer una tasa de aprendizaje específica pasando un parámetro al método apply.
    fgMask = backSub.apply(frame)
    
    # Escribimos sobre la imagen el número de frame procesado
    cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    
    # mostramos frame original e imagen binaria background/foreground
    cv.imshow('Frame', frame)
    cv.imshow('FG Mask', fgMask)
    
    # Corresmos hasta que termine o apriete escape
    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break

cv.destroyAllWindows()
capture.release()

29.97002997002997 914 30.497133333333334


## Implementación de algoritmo Naive Background Subtraction

### Funciones auxiliares

In [137]:
def procesamientoMorfologico(thresh):
    
    # Creamos un elemento estructurante y aplicamos operaciones morfologicas
    kernel = np.ones((5,5), np.uint8)

    # Aplicamos una serie de iteraciones de apertura
    opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations=2)

    # Búsqueda del área de objeto (foreground) segura.
    # Transformación de distancia (distancia de cada píxel al cero más cercano)
    #------------------------------------------------------------------------------------
    # Conviene cerrar primero la figura para que no haya agujeros (ceros) dentro de los objetos que afecten la transformada
    closing = cv.morphologyEx(opening, cv.MORPH_CLOSE, kernel, iterations=2)
    # Ahora sí aplicamos la transformación de distancia
    dist_transform = cv.distanceTransform(closing, cv.DIST_L2, maskSize=5)
    # Área de objeto (sure foreground) segura. Zona blanca!
    ret, sure_fg = cv.threshold(dist_transform, 0.08*dist_transform.max(), 255, 0)
    # Búsqueda de la región no definida
    sure_fg = np.uint8(sure_fg)
    
    return sure_fg

### Funcion para deteccion de fondo mediante método Naive Background Subtraction

In [145]:
def naiveBS(filename, frame, frame_count, median, N, pm):
    
    #-------------------------
    # Tamaño del fame actual
    h, w, _ = frame.shape
    
    # Si la mediana que la paso es "None" la calculo, sino utilizo la que paso como argumento
    if median is None:
    
        # Lista de capturas --> Repito N veces el objecto "capture" para no modificar el objeto original
        CAP = []
        for i in range(N):
            CAP.append(cv.VideoCapture(filename))

        #-------------------------
        # Genero una lista de N numeros aleatorios con los que selecciono los frames asociados
        indexes = np.random.randint(0, frame_count, N)
        rf_r = np.zeros((h,w,N))
        rf_g = np.zeros((h,w,N))
        rf_b = np.zeros((h,w,N))
        for i, idx in enumerate(indexes):
            cap = CAP[i]
            cap.set(cv.CAP_PROP_POS_FRAMES, idx)
            randomFrame = cap.read()
            rf_r[:, :, i] = randomFrame[1][:, :, 0]
            rf_g[:, :, i] = randomFrame[1][:, :, 1]
            rf_b[:, :, i] = randomFrame[1][:, :, 2]

        #-------------------------
        # Calculo la mediana de los frames aleatorios (canal a canal)
        median_r = np.median(rf_r, axis=2)
        median_g = np.median(rf_g, axis=2)
        median_b = np.median(rf_b, axis=2)
        median = np.dstack((median_r, median_g, median_b))
    
    #-------------------------
    # Resto la mediana al frame actual para obtener la máscara de foreground y casteo a float32
    diff = frame - median
    diff = np.float32(diff)

    #-------------------------
    # Paso a escala de grises, casteo a uint8, y binarizo
    gray = cv.cvtColor(diff, cv.COLOR_RGB2GRAY)
    _, fgMask = cv.threshold(gray.astype("uint8"), 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU) #cv.THRESH_OTSU) #cv.THRESH_BINARY+

    #-------------------------
    # Procesamiento morfológico
    if pm==1:
        fgMask = procesamientoMorfologico(fgMask)
    
    
    return fgMask, median 

### Corremos el algoritmo implementado

In [148]:
# Parámetros para correr el método Naive
N = 3 #Cantidad de frames aleatorios a seleccionar
interval = 2.5 #Intervalo de tiempo (en segundos) donde la mediana calculada se mantiene constante

# Abro el archivo
#-------------------
filename = 'slow_traffic_small.mp4'
capture = cv.VideoCapture(filename)

# Obtengo datos del video
fps = capture.get(cv.CAP_PROP_FPS)
frame_count = int(capture.get(cv.CAP_PROP_FRAME_COUNT))
duration = frame_count/fps

# Inicializo la mediana como "None" para calcularla en el primer paso
median = None

# Indico si deseo aplicar procesamiento morfologico o no
pm = 1

# Corremos la sustraccion
#------------------------
cont = 1
while True:
    
    # Tiempo transcurrido
    elapsed_time = 1/fps*cont
    aux = np.mod(cont, int(fps*interval))
    
    # Leemos un frame
    ret, frame = capture.read()
    if frame is None:
        break
    
    # Si llego al limite del tiempo definido por el intervalo, vuelvo a definir la mediana como "None"
    if np.mod(cont, int(fps*interval))==0:
        median = None
    
    # Obtengo mediana y máscara
    fgMask, median = naiveBS(filename, frame, frame_count, median, N, pm) 
    
    # Escribimos sobre la imagen el número de frame procesado
    cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(frame, str(capture.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0))
    
    # Mostramos frame original e imagen binaria background/foreground
    cv.imshow('Frame', frame)
    cv.imshow('FG Mask', fgMask)
    
    # Corremos hasta que termine o apriete escape
    keyboard = cv.waitKey(int(duration))
    if keyboard == 'q' or keyboard == 27:
        break
    
    # Actualizo contador
    cont += 1
    
cv.destroyAllWindows()
capture.release()

29.97002997002997 914 30.497133333333334


## Conclusiones

xxxxx